In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

def graph_kmeans(data, means, assignments):
    colors = cm.rainbow(np.linspace(0, 1, len(means)))
    data = np.array(data)
    assignments = np.array(assignments)
    fig, ax = plt.subplots(1,1, figsize=(8,8))
    
    # temp plot to get limits
    ax.plot(*zip(*data), color='k', marker='o', markersize=5, linestyle='None')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    

    if len(assignments):
        ax.lines.pop()
        if len(means) > 2:
            voronoi_plot_2d(Voronoi(means), ax=ax, show_vertices=False, show_points=False)
    for i, pnt in enumerate(means):
        clr = colors[i]
        ax.plot(*pnt, color=clr, marker='*', markersize=10, linestyle='None')
        ax.plot(*zip(*data[assignments == i]), color=clr, marker='o', markersize=5, linestyle='None')

    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    return fig

The following cell is copied and adapted from https://github.com/joelgrus/data-science-from-scratch/blob/master/code-python3/clustering.py, the github repository for the book “Data Science from Scratch by Joel Grus (O’Reilly). Copyright 2015 Joel Grus, 978-1-4919-0142-7.”

In [ ]:
from linear_algebra import squared_distance, vector_mean, distance
import math, random
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

class KMeans:
    """performs k-means clustering"""

    def __init__(self, k):
        self.k = k          # number of clusters
        self.means = None   # means of clusters

    def classify(self, input):
        """return the index of the cluster closest to the input"""
        return min(range(self.k),
                   key=lambda i: squared_distance(input, self.means[i]))

    def train(self, inputs, plot=True):

        self.means = random.sample(inputs, self.k)
        assignments = []
        if plot:
            fig = graph_kmeans(inputs, self.means, assignments)
            fig.suptitle("k={}, iteration={}".format(self.k, 0))
            fig.savefig("kmeans_k{}iter{}.png".format(self.k, 0))
            plt.close()
        iteration = 1

        while True:
            # Find new assignments
            new_assignments = list(map(self.classify, inputs))
            if plot:
                fig = graph_kmeans(inputs, self.means, new_assignments)
                fig.suptitle("k={}, iteration={}".format(self.k, iteration))
                fig.savefig("kmeans_k{}iter{}.png".format(self.k, iteration))
                plt.close()
            
            # If no assignments have changed, we're done.
            if assignments == new_assignments:
                return assignments

            # Otherwise keep the new assignments,
            assignments = new_assignments

            for i in range(self.k):
                i_points = [p for p, a in zip(inputs, assignments) if a == i]
                # avoid divide-by-zero if i_points is empty
                if i_points:
                    self.means[i] = vector_mean(i_points)
            iteration += 1

def squared_clustering_errors(inputs, k, plot=False):
    """finds the total squared error from k-means clustering the inputs"""
    clusterer = KMeans(k)
    clusterer.train(inputs, plot)
    means = clusterer.means
    assignments = list(map(clusterer.classify, inputs))

    return sum(squared_distance(input,means[cluster])
               for input, cluster in zip(inputs, assignments))

def plot_squared_clustering_errors(scatterplot=False):

    ks = range(1, len(inputs) + 1)
    errors = [squared_clustering_errors(inputs, k, scatterplot) for k in ks]

    plt.plot(ks, errors)
    plt.xticks(ks)
    plt.xlabel("k")
    plt.ylabel("total squared error")
    plt.show()

def run_kmeans(inputs, plot=False):


    random.seed(0) # so you get the same results as me
    clusterer = KMeans(3)
    clusterer.train(inputs, plot)
    print("3-means:")
    print(clusterer.means)
    print()

    random.seed(0)
    clusterer = KMeans(2)
    clusterer.train(inputs, plot)
    print("2-means:")
    print(clusterer.means)
    print()

    errors = [(k, squared_clustering_errors(inputs, k)) for k in range(1, min(len(inputs) + 1, 20))]
    plt.plot(*zip(*errors))
    plt.xlabel("k")
    plt.ylabel("J")
       
        
#
# hierarchical clustering
#

def is_leaf(cluster):
    """a cluster is a leaf if it has length 1"""
    return len(cluster) == 1

def get_children(cluster):
    """returns the two children of this cluster if it's a merged cluster;
    raises an exception if this is a leaf cluster"""
    if is_leaf(cluster):
        raise TypeError("a leaf cluster has no children")
    else:
        return cluster[1]

def get_values(cluster):
    """returns the value in this cluster (if it's a leaf cluster)
    or all the values in the leaf clusters below it (if it's not)"""
    if is_leaf(cluster):
        return cluster # is already a 1-tuple containing value
    else:
        return [value
                for child in get_children(cluster)
                for value in get_values(child)]

def cluster_distance(cluster1, cluster2, distance_agg=min):
    """finds the aggregate distance between elements of cluster1
    and elements of cluster2"""
    return distance_agg([distance(input1, input2)
                        for input1 in get_values(cluster1)
                        for input2 in get_values(cluster2)])

def get_merge_order(cluster):
    if is_leaf(cluster):
        return float('inf')
    else:
        return cluster[0] # merge_order is first element of 2-tuple

def bottom_up_cluster(inputs, distance_agg=min):
    # start with every input a leaf cluster / 1-tuple
    clusters = [(input,) for input in inputs]

    # as long as we have more than one cluster left...
    while len(clusters) > 1:
        # find the two closest clusters
        c1, c2 = min([(cluster1, cluster2)
                     for i, cluster1 in enumerate(clusters)
                     for cluster2 in clusters[:i]],
                     key=lambda p: cluster_distance(p[0], p[1], distance_agg))

        # remove them from the list of clusters
        clusters = [c for c in clusters if c != c1 and c != c2]

        # merge them, using merge_order = # of clusters left
        merged_cluster = (len(clusters), [c1, c2])

        # and add their merge
        clusters.append(merged_cluster)

    # when there's only one cluster left, return it
    return clusters[0]


def generate_clusters(base_cluster, num_clusters):
    # start with a list with just the base cluster
    clusters = [base_cluster]

    # as long as we don't have enough clusters yet...
    while len(clusters) < num_clusters:
        # choose the last-merged of our clusters
        next_cluster = min(clusters, key=get_merge_order)
        # remove it from the list
        clusters = [c for c in clusters if c != next_cluster]
        # and add its children to the list (i.e., unmerge it)
        clusters.extend(get_children(next_cluster))

    # once we have enough clusters...
    return clusters


In [ ]:
# Data Science From Scratch example
# Cluster graphs will be saved to current working directory
run_kmeans(inputs = [[-14,-5],[13,13],[20,23],[-19,-11],[-9,-16],[21,27],[-49,15],[26,13],[-46,5],[-34,-1],[11,15],[-49,0],[-22,-16],[19,28],[-12,-8],[-13,-19],[-41,8],[-11,-6],[-25,-9],[-18,-3]]
)

In [ ]:
# Portland public art example
# Note that cluster graphs will overwrite those produced above - move them to another directory to save
art = pd.read_excel("public_art.xlsx").dropna()
inputs = [ tuple(x) for x in art[["lng", "lat"]].values ]
run_kmeans(inputs, plot=True)

In [ ]:
# Scatter plot of public art locations
art.plot("lng", "lat", "scatter", figsize=(8,8))

In [ ]:
# Scatter plot of data science from scratch example
inputs = [[-14,-5],[13,13],[20,23],[-19,-11],[-9,-16],[21,27],[-49,15],[26,13],[-46,5],[-34,-1],[11,15],[-49,0],[-22,-16],[19,28],[-12,-8],[-13,-19],[-41,8],[-11,-6],[-25,-9],[-18,-3]]
fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(*zip(*inputs))
for label, (x,y) in enumerate(inputs):
    ax.annotate(label, xy=(x,y), fontsize=12)

In [ ]:
# Data science from scratch example
inputs = np.array([[-14,-5],[13,13],[20,23],[-19,-11],[-9,-16],[21,27],[-49,15],[26,13],[-46,5],[-34,-1],[11,15],[-49,0],
                   [-22,-16],[19,28],[-12,-8],[-13,-19],[-41,8],[-11,-6],[-25,-9],[-18,-3]])
fig, ax = plt.subplots(figsize=(16,10))
clusterlinkage = linkage(inputs)

# dendrogram
dn = dendrogram(clusterlinkage, ax=ax)
ax.set_xlabel("data point index")
ax.set_ylabel("distance")

In [ ]:
# scatter plot
fig, axes = plt.subplots(6, 3, figsize=(16,30))
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
markers = ['o', '*', 'x']
colmark = [(c, m) for c in colors for m in markers]
for n_clusters in range(2,len(inputs)):
    cluster_identity = fcluster(clusterlinkage, n_clusters, 'maxclust')
    for c in range(1, n_clusters+1):
        cluster = inputs[cluster_identity == c]
        ax = axes.flatten()[n_clusters-2]
        ax.plot(*zip(*cluster), marker=colmark[c][1], linestyle='None', color=colmark[c][0])
        ax.set_title("{} Clusters".format(n_clusters))